## World Development Indicators - Analytics

In [1]:
import pandas as pd
import numpy as np
import time

datos = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)
paises = ['COL']

preprocessing = 'imput'
search = 'original'
year_init = 2012
year_range = 5
look_back = 3
years = range(year_init-year_range+1,year_init+1)[::-1]

agricultura     = open("Indicadores/iagricultura.txt").read().split(',')
ambiente        = open("Indicadores/iambiente.txt").read().split(',')
ayuda           = open("Indicadores/iayuda.txt").read().split(',')
ciencia         = open("Indicadores/iciencia.txt").read().split(',')
clima           = open("Indicadores/iclima.txt").read().split(',')
comercio        = open("Indicadores/icomercio.txt").read().split(',')
deuda           = open("Indicadores/ideuda.txt").read().split(',')
economia        = open("Indicadores/ieconomia.txt").read().split(',')
educacion       = open("Indicadores/ieducacion.txt").read().split(',')
energia         = open("Indicadores/ienergia.txt").read().split(',')
finanzas        = open("Indicadores/ifinanzas.txt").read().split(',')
genero          = open("Indicadores/igenero.txt").read().split(',')
infraestructura = open("Indicadores/iinfraestructura.txt").read().split(',')
pobreza         = open("Indicadores/ipobreza.txt").read().split(',')
privado         = open("Indicadores/iprivado.txt").read().split(',')
publico         = open("Indicadores/ipublico.txt").read().split(',')
salud           = open("Indicadores/isalud.txt").read().split(',')
social          = open("Indicadores/isocial.txt").read().split(',')
trabajo         = open("Indicadores/itrabajo.txt").read().split(',')
urbano          = open("Indicadores/iurbano.txt").read().split(',')


conjunto_nombre = ['Agricultura','Ambiente','Ayuda','Ciencia','Clima','Comercio','Deuda','Economia','Educacion',
                   'Energia','Finanzas','Genero','Infraestructura','Pobreza','Privado','Publico','Salud','Social',
                   'Trabajo','Urbano']

conjunto = [agricultura,ambiente,ayuda,ciencia,clima,comercio,deuda,economia,educacion,energia,finanzas,genero,
            infraestructura,pobreza,privado,publico,salud,social,trabajo,urbano]

datos.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Antigua and Barbuda,ATG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,126.144000
1,Antigua and Barbuda,ATG,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,88.237117
2,Antigua and Barbuda,ATG,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,7.779958
3,Antigua and Barbuda,ATG,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,80.457159
4,Antigua and Barbuda,ATG,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,32.920000


In [2]:
def r2(y_true, y_predict):
    from sklearn.metrics import r2_score
    return r2_score(y_true, y_predict)

In [3]:
def tabla_base(indicadores):
    tab = pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False])
    return tab

In [4]:
def tabla_2_base(indicadores,look_back):
    temp_table = []
    for i in range(look_back):      
        temp_years = range(year_init-year_range-i+1,year_init-i+1)[::-1]
        temp_table.append(pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,temp_years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False]))
    return pd.DataFrame(np.column_stack(temp_table))

In [5]:
def estimator_Universal(estimador, X_train, X_test, y_train, y_test):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.svm import SVR
    
    if(search=='original'):
        if(estimador=='DTR'):
            estimator = DecisionTreeRegressor()
        if(estimador=='RFR'):
            estimator = RandomForestRegressor(n_jobs=-1)
        if(estimador=='SVR'):
            estimator = SVR()
    else:
        best_params = SearchCV_Universal(estimador, search, X_train, y_train)       

        if(estimador=='DTR'):
            estimator = DecisionTreeRegressor().set_params(**best_params)
        if(estimador=='RFR'):
            estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
        if(estimador=='SVR'):
            estimator = SVR().set_params(**best_params)
        
    estimator.fit(X_train,y_train)
    y_predict = estimator.predict(X_test)
        
    return r2(y_test,y_predict)

In [6]:
def Preprocess(tab1,tab2,y_indicator):
    from sklearn.preprocessing import Imputer
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split

    #Eliminamos las columnas de NaN descartando Indicadores que no tienen regristros para ningún pais y año deseados
    tab1 = tab1.dropna(how='all',axis=1)
    tab2 = tab2.dropna(how='all',axis=1)

    if(preprocessing=='zeros'):
        #Imputamos los NaN por Zero
        tab1 = tab1.fillna(0)      

    if(preprocessing=='imput'):
        #Imputamos los NaN por la media de cada Indicador respectivamente      
        impute=Imputer(missing_values="NaN",strategy='mean',axis=0)
        impute.fit(tab1)
        tab1 = pd.DataFrame(impute.transform(tab1))

    #Fusionamos la tabla_1 y el indicador y de la tabla_2
    tab_fusion = pd.DataFrame(np.column_stack((np.array(tab1)[:,:],np.array(tab2)[:,y_indicator])))

    #Eliminamos las filas Si el valor a predecir es NaN
    tab_fusion = tab_fusion.dropna(subset=[tab_fusion.iloc[:,-1].name])

    # Asignamos X e y, eliminando los indicadores que se correlacionen más con el indicador a predecir (coeficiente > 0.7)
    tab_fusion_corr = tab_fusion.corr()
    X = tab_fusion.drop(tab_fusion_corr[tab_fusion_corr.iloc[:,-1] > 0.7].index, axis=1)
    y = tab_fusion.iloc[:,-1]

    # Normalizamos los datos
    sc = StandardScaler()
    tab_fusion_norm = sc.fit_transform(np.column_stack([X,y]))
    X = tab_fusion_norm[:,:-1]
    y = tab_fusion_norm[:,-1]

    # Separamos Train y Test respectivamente para X e y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    return X_train, X_test, y_train, y_test

In [7]:
def iter_Splitter_Optimus(tab1,tab2): 
    R2_global = list()
    for i in range(0,np.shape(tab2.dropna(how='all',axis=1))[1]):

        X_train, X_test, y_train, y_test = Preprocess(tab1, tab2, i)

        result = estimator_Universal('DTR', X_train, X_test, y_train, y_test)

        if(result < 0.9): 
            temp = estimator_Universal('SVR', X_train, X_test, y_train, y_test)
            if(temp < 0.9): 
                temp2 = estimator_Universal('RFR', X_train, X_test, y_train, y_test)
                if (temp2 > temp): 
                    result = temp2
            if(temp > result): 
                result = temp
        print i , result
        R2_global.append(result)
    return R2_global

In [8]:
def SearchCV_Universal(estimador, search, X_train, y_train):
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import ShuffleSplit
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.svm import SVR
        
    if(estimador=='DTR'):
        estimator  = DecisionTreeRegressor()
        param_grid = {  'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                        'splitter': ['best', 'random']
                     }
        
    if(estimador=='RFR'):
        estimator  = RandomForestRegressor()       
        param_grid = { 
                        "n_estimators"      : [10,20,30,40],
                        "max_features"      : ["auto", "sqrt", "log2"],
                        "min_samples_split" : [2,4,8],
                        "bootstrap": [True, False],
                     }
    if(estimador=='SVR'):
        estimator  = SVR()
        param_grid ={
                        'gamma'  : ['auto', 1e-3, 1e-4],
                        'C'      : [1, 10, 100, 1000],            
                    }

    if (search=='random'):
        grid = RandomizedSearchCV(estimator, param_grid, n_jobs=-1, cv=ShuffleSplit(test_size=0.2))
    if (search=='grid'):
        grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=ShuffleSplit(test_size=0.2))
        
    grid.fit(X_train, y_train)
    
    return grid.best_params_

In [9]:
def filtro(indicadores_1,indicadores_2):
    if(indicadores_1!=indicadores_2):
        df = pd.DataFrame(indicadores_2)
        indicadores_2 = np.array(df.loc[~df.ix[:,0].isin(indicadores_1)]).flatten()
    return indicadores_2

In [10]:
def iterador_global(indicadores_1,indicadores_2,look_back): 
    
    tab1 = tabla_2_base(indicadores_1,look_back)
    tab2 = tabla_base(filtro(indicadores_1,indicadores_2))
    
    return iter_Splitter_Optimus(tab1,tab2)

In [15]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

y_indicator = 3
tab1 = tabla_2_base(conjunto[0],1)
tab2 = tabla_base(filtro(conjunto[0],conjunto[1]))

#Eliminamos las columnas de NaN descartando Indicadores que no tienen regristros para ningún pais y año deseados
tab1 = tab1.dropna(how='all',axis=1)
tab2 = tab2.dropna(how='all',axis=1)

if(preprocessing=='zeros'):
    #Imputamos los NaN por Zero
    tab1 = tab1.fillna(0)      

if(preprocessing=='imput'):
    #Imputamos los NaN por la media de cada Indicador respectivamente      
    impute=Imputer(missing_values="NaN",strategy='mean',axis=0)
    impute.fit(tab1)
    tab1 = pd.DataFrame(impute.transform(tab1))

#Fusionamos la tabla_1 y el indicador y de la tabla_2
tab_fusion = pd.DataFrame(np.column_stack((np.array(tab1)[:,:],np.array(tab2)[:,y_indicator])))

tab_fusion

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,87.9,49.806023,0.944369,38.411537,426176.0,66.260984,42975.0,82.469443,20228.05,2.422891,...,1165592.23,116.53,1.689950,11437562.0,24.397,-0.131803,19.5,46.8,1141748.0,8.570292e+09
1,89.5,48.960221,0.944369,37.603425,417210.0,66.260984,42975.0,82.469443,20228.05,2.499631,...,1136557.50,110.30,1.643082,11452647.0,24.679,-0.090314,19.2,46.1,1141750.0,5.124973e+09
2,91.1,48.114419,0.944369,38.308247,425030.0,65.659383,43791.0,80.011853,20116.50,3.505426,...,1039816.70,106.20,1.433078,11462995.0,24.964,-0.044211,21.6,49.7,1141750.0,6.821874e+09
3,89.5,48.960221,0.944369,38.341595,425400.0,66.260984,42975.0,82.469443,20228.05,3.602386,...,1160333.00,106.34,1.423164,11468064.0,25.251,0.001570,20.1,53.7,1141750.0,9.973594e+09
4,89.5,48.960221,0.944369,38.408292,426140.0,66.862586,42159.0,84.927034,20339.60,3.524266,...,1182990.00,116.31,1.470032,11467884.0,25.540,0.047701,20.1,56.6,1141750.0,6.014233e+09


In [29]:
def porcent_result_reg(df,indicadores_1,indicadores_2):
    df = pd.DataFrame(df)
    df[df < 0] = 0.0
    porcent = df.mean().values[0]
    result = np.array(df.values)   
    reg = len(filtro(indicadores_1,indicadores_2))
    return porcent,result,reg

## Iteramos todos los Conjuntos de Indicadores

In [35]:
start_time = time.time()
resultado_global = []

for i in range(len(conjunto)):
    for j in range(len(conjunto)):
        for k in range(1,look_back+1):
            start_time2 = time.time()
            porcent , results, reg = porcent_result_reg(iterador_global(conjunto[i],conjunto[j],k),conjunto[i],conjunto[j])
            resultado_global.append([conjunto_nombre[i],conjunto_nombre[j],k,reg,porcent,"%s" % (time.time() - start_time2),results])
            pd.DataFrame(resultado_global, columns=["Base","Target","Look Back","Reg","%","Time","Results"]).to_csv('colombia_original_log.csv')


df_global = pd.DataFrame(resultado_global, columns=["Base","Target","Look Back","Reg","%","Time","Results"])
print("--- %s seconds ---" % (time.time() - start_time))
df_global

0 0.0
1 0.0


ValueError: Found array with 0 sample(s) (shape=(0, 43)) while a minimum of 1 is required.

In [45]:
for i in range(0,1):
    print i

0


In [47]:
for j in range(1,20):
    print j

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
